# Lab 1: Implementing self-editing memory from scratch

Hi, My boyfriend's name is Virat

A - Hi

Write a poem on my boyfried's name

My boy fried's name is Sachin

AI Agents -> tools

Tool -> Managing / editing the memory

## Section 0: Setup OpenAI

In [1]:
!pip install python_dotenv==1.0.1 openai==1.45.1 letta==0.1.4 crewai-tools==0.12.0


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.5/131.5 kB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of embedchain to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of locust to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.8 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of langchain-cohere to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of llama-index-indices-managed-llama-cloud to determine 

In [1]:
from google.colab import userdata
openai_api_key = userdata.get('OPEN_API_KEY')


In [2]:
from openai import OpenAI
import os

client = OpenAI(
    api_key=openai_api_key
)

## Section 1: Breaking down the LLM context window
### A simple agent's context window

In [3]:
model = "gpt-4o-mini"

In [4]:
system_prompt = "You are a helpful chatbot."

In [5]:
# Make the completion request with the tool usage
chat_completion = client.chat.completions.create(
    model=model,
    messages=[
        # system prompt: always included in the context window
        {"role": "system", "content": system_prompt},
        # chat history (evolves over time)
        {"role": "user", "content": "What is my name?"},
    ]
)
chat_completion.choices[0].message.content

"I'm sorry, but I don't know your name. You can tell me if you'd like!"

### Adding memory to the context


In [6]:
agent_memory = {"human": "Name: Sachin"}
system_prompt = "You are a helpful chatbot. " \
+ "You have a section of your context called [MEMORY] " \
+ "that contains information relevant to your conversation"

In [7]:
import json


chat_completion = client.chat.completions.create(
    model=model,
    messages=[
        # system prompt
        {"role": "system", "content": system_prompt + "[MEMORY]\n" + \
         json.dumps(agent_memory)},
        # chat history
        {"role": "user", "content": "What is my name?"},
    ],
)
chat_completion.choices[0].message.content

'Your name is Sachin.'

## Section 2: Modifing the memory with tools

### Defining a memory editing tool


In [8]:
agent_memory = {"human": "", "agent": ""}

def core_memory_save(section: str, memory: str):
    agent_memory[section] += '\n'
    agent_memory[section] += memory

In [9]:
agent_memory

{'human': '', 'agent': ''}

In [10]:
core_memory_save("human", "The human's name is Rahul")

In [11]:
agent_memory

{'human': "\nThe human's name is Rahul", 'agent': ''}

In [12]:
# tool description
core_memory_save_description = "Save important information about you," \
+ "the agent or the human you are chatting with."

# arguments into the tool (generated by the LLM)
# defines what the agent must generate to input into the tool
core_memory_save_properties = \
{
    # arg 1: section of memory to edit
    "section": {
        "type": "string",
        "enum": ["human", "agent"],
        "description": "Must be either 'human' " \
        + "(to save information about the human) or 'agent'" \
        + "(to save information about yourself)",
    },
    # arg 2: memory to save
    "memory": {
        "type": "string",
        "description": "Memory to save in the section",
    },
}

# tool schema (passed to OpenAI)
core_memory_save_metadata = \
    {
        "type": "function",
        "function": {
            "name": "core_memory_save",
            "description": core_memory_save_description,
            "parameters": {
                "type": "object",
                "properties": core_memory_save_properties,
                "required": ["section", "memory"],
            },
        }
    }

In [13]:
agent_memory = {"human": ""}
system_prompt = "You are a helpful chatbot. " \
+ "You have a section of your context called [MEMORY] " \
+ "that contains information relevant to your conversation"

chat_completion = client.chat.completions.create(
    model=model,
    messages=[
        # system prompt
        {"role": "system", "content": system_prompt},
        # memory
        {"role": "system", "content": "[MEMORY]\n" + json.dumps(agent_memory)},
        # chat history
        {"role": "user", "content": "My name is Virat"},
    ],
    # tool schemas
    tools=[core_memory_save_metadata]
)
response = chat_completion.choices[0]
response

Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_XQeFfk2MOgqhMp28zzhnDtzG', function=Function(arguments='{"section":"human","memory":"The human\'s name is Virat."}', name='core_memory_save'), type='function')]))

### Executing the tool


In [14]:
arguments = json.loads(response.message.tool_calls[0].function.arguments)
arguments

{'section': 'human', 'memory': "The human's name is Virat."}

In [16]:
# run the function with the specified arguments
core_memory_save(**arguments)

In [17]:
agent_memory

{'human': "\nThe human's name is Virat."}

### Running the next agent step


In [18]:
chat_completion = client.chat.completions.create(
    model=model,
    messages=[
        # system prompt
        {"role": "system", "content": system_prompt},
        # memory
        {"role": "system", "content": "[MEMORY]\n" + json.dumps(agent_memory)},
        # chat history
        {"role": "user", "content": "what is my name"},
    ],
    tools=[core_memory_save_metadata]
)
response = chat_completion.choices[0]
response.message

ChatCompletionMessage(content='Your name is Virat.', refusal=None, role='assistant', function_call=None, tool_calls=None)

## Implementing an agentic loop


In [21]:
system_prompt_os = system_prompt \
+ "\n. You must either call a tool (core_memory_save) or" \
+ "write a response to the user. " \
+ "Do not take the same actions multiple times!" \
+ "When you learn new information, make sure to always" \
+ "call the core_memory_save tool."

In [22]:
def agent_step(user_message):

    # prefix messages with system prompt and memory
    messages = [
        # system prompt
        {"role": "system", "content": system_prompt_os},
        # memory
        {
            "role": "system",
            "content": "[MEMORY]\n" + json.dumps(agent_memory)
        },
    ]

    # append the most recent message
    messages.append({"role": "user", "content": user_message})

    # agentic loop
    while True:
        chat_completion = client.chat.completions.create(
            model=model,
            messages=messages,
            tools=[core_memory_save_metadata]
        )
        response = chat_completion.choices[0]

        # update the messages with the agent's response
        messages.append(response.message)

        # if NOT calling a tool (responding to the user), return
        if not response.message.tool_calls:
            return response.message.content

        # if calling a tool, execute the tool
        else:
            print("TOOL CALL:", response.message.tool_calls[0].function)

            # parse the arguments from the LLM function call
            arguments = json.loads(
                response.message.tool_calls[0].function.arguments
            )

            # run the function with the specified arguments
            core_memory_save(**arguments)

            # add the tool call response to the message history
            messages.append({
                "role": "tool",
                "tool_call_id": response.message.tool_calls[0].id,
                "name": "core_memory_save",
                "content": f"Updated memory: {json.dumps(agent_memory)}"
            })

In [23]:
agent_step("my name is Sourav.")

TOOL CALL: Function(arguments='{"section":"human","memory":"The human\'s name is Sourav."}', name='core_memory_save')


'Got it, Sourav! How can I assist you today?'

In [ ]:
# Try some prompts of your own!